In [2]:
import numpy as np
import pandas as pd
import os
print(os.listdir("data"))

['heart.csv']


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.metrics import confusion_matrix

In [5]:
heart_path = "data/heart.csv"
heart_data = pd.read_csv(heart_path)

In [6]:
heart_data = pd.get_dummies(heart_data)
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
X = heart_data.drop(["target"], axis=1)
y = heart_data["target"]

## Random Forest

In [9]:

rf_model = RandomForestClassifier(n_estimators=100)
rf_predictions = cross_val_predict(rf_model, X, y, cv=5)
print(confusion_matrix(y, rf_predictions))
rf_scores = cross_val_score(rf_model, X, y, scoring="recall", cv=5)
print("recall:", rf_scores.mean())

[[108  30]
 [ 26 139]]
recall: 0.8484848484848486


## SVM

In [10]:
X["age"] = X["age"].map(lambda x: (x - X["age"].min()) / (X["age"].max() - X["age"].min()))
X["trestbps"] = X["trestbps"].map(lambda x: (x - X["trestbps"].min()) / (X["trestbps"].max() - X["trestbps"].min()))
X["chol"] = X["chol"].map(lambda x: (x - X["chol"].min()) / (X["chol"].max() - X["chol"].min()))
X["thalach"] = X["thalach"].map(lambda x: (x - X["thalach"].min()) / (X["thalach"].max() - X["thalach"].min()))
X["oldpeak"] = X["oldpeak"].map(lambda x: (x - X["oldpeak"].min()) / (X["oldpeak"].max() - X["oldpeak"].min()))

In [11]:
# Support Vector Machine
svc_model = SVC(gamma="auto")
svc_predictions = cross_val_predict(svc_model, X, y, cv=5)
print(confusion_matrix(y, svc_predictions))
svc_scores = cross_val_score(svc_model, X, y, scoring="recall", cv=5)
print("recall:", svc_scores.mean())

[[ 99  39]
 [ 10 155]]
recall: 0.9393939393939394


## MLP

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import lite
import time

In [ ]:
epochs = 100
batch_size = 25
learning_rate = 0.0001

mlp_model = Sequential()

#First Hidden Layer
mlp_model.add(Dense(512, activation='relu', name='hidden_layer_1', input_dim = X.shape[1]))

#Second  Hidden Layer
mlp_model.add(Dense(512, activation='relu', name='hidden_layer_2'))

#Output Layer
output = mlp_model.add(Dense(1, activation='sigmoid', name='predictions_output'))

mlp_model.summary()

In [ ]:
start_time = time.time()

mlp_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = mlp_model.fit(X, y, batch_size, epochs)


print("> Temp d'exécution: %s secondes ---" % (time.time() - start_time))

In [ ]:
# Convert the model in tflite
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

# Save le model
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Print input shape and type
inputs = interpreter.get_input_details()
print('{} input(s):'.format(len(inputs)))
for i in range(0, len(inputs)):
    print('{} {}'.format(inputs[i]['shape'], inputs[i]['dtype']))

# Print output shape and type
outputs = interpreter.get_output_details()
print('\n{} output(s):'.format(len(outputs)))
for i in range(0, len(outputs)):
    print('{} {}'.format(outputs[i]['shape'], outputs[i]['dtype']))


In [ ]:
#Inference data
existing_data = [[63, 45, 233, 150, 2.3, 0, 0,	1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]]
new_data = [[63, 45, 233, 150, 2.3, 0, 0,	1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1]]
